# 랭체인 스타일로 프롬프트 작성하는 방법

이 노트북에서는 LangChain을 사용하여 프롬프트를 구조화하고 템플릿화하는 방법을 알아보겠습니다.

## 1. LLM 설정

먼저 기본적인 LLM을 설정합니다.


In [1]:
from langchain_ollama import ChatOllama


# Ollama를 이용한 로컬 LLM 설정
llm = ChatOllama(model="gemma3:12b")

# 에러를 발생시키는 코드
# llm.invoke(0)

## 2. 기본 프롬프트 템플릿 사용

PromptTemplate을 사용하여 변수가 포함된 템플릿을 만들 수 있습니다.


In [5]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

# 변수가 포함된 템플릿 정의
prompt_template = PromptTemplate(
    template="What is the capital of {country}? Return the name of the city only",
    input_variables=["country"],
)

# 아래처럼 해도 위와 같은 결과를 얻을 수 있습니다.
prompt_template.from_template("What is the capital of {country}? Return the name of the city only")

# 템플릿에 변수 값을 주입
prompt = prompt_template.invoke({"country": "France"})

print(prompt)

# LLM에 프롬프트 전달
llm.invoke(prompt_template.invoke({"country": "France"}))

text='What is the capital of France?'


AIMessage(content='The capital of France is Paris.', additional_kwargs={}, response_metadata={'model': 'llama3.2:1b', 'created_at': '2025-02-02T13:19:03.64102Z', 'done': True, 'done_reason': 'stop', 'total_duration': 196228500, 'load_duration': 31930459, 'prompt_eval_count': 32, 'prompt_eval_duration': 73000000, 'eval_count': 8, 'eval_duration': 90000000, 'message': Message(role='assistant', content='', images=None, tool_calls=None)}, id='run-81d2645d-5d5d-4a12-b6f2-147b53f44cd6-0', usage_metadata={'input_tokens': 32, 'output_tokens': 8, 'total_tokens': 40})

## 3. 메시지 기반 프롬프트 작성

시스템 메시지, 사용자 메시지, AI 메시지를 조합하여 대화 형식의 프롬프트를 만들 수 있습니다.


  

In [12]:
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage

# 대화 형식의 메시지 리스트 생성
message_list = [
    SystemMessage(content="You are a helpful assistant!"),
    HumanMessage(content="What is the capital of France?"),
    AIMessage(content="The capital of France is Paris."),
    HumanMessage(content="What is the capital of Germany?"),
    AIMessage(content="The capital of Germany is Berlin."),
    HumanMessage(content="What is the capital of {country}?"),
]

llm.invoke(message_list)

AIMessage(content="If you're referring to a specific country, I'd be happy to try and help. Please provide the name of the country, and I'll do my best to tell you its capital.", additional_kwargs={}, response_metadata={'model': 'llama3.2:1b', 'created_at': '2025-02-02T13:24:19.600569Z', 'done': True, 'done_reason': 'stop', 'total_duration': 931122208, 'load_duration': 31664958, 'prompt_eval_count': 87, 'prompt_eval_duration': 142000000, 'eval_count': 39, 'eval_duration': 755000000, 'message': Message(role='assistant', content='', images=None, tool_calls=None)}, id='run-c5b88f61-5ba2-49a9-bcfc-d33ae743d7ca-0', usage_metadata={'input_tokens': 87, 'output_tokens': 39, 'total_tokens': 126})

## 4. ChatPromptTemplate 사용

ChatPromptTemplate을 사용하면 메시지 기반 프롬프트를 더 쉽게 만들 수 있습니다.


In [16]:
from langchain_core.prompts import ChatPromptTemplate

# 채팅 형식의 프롬프트 템플릿 생성
chat_prompt_template = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant!"),
    ("human", "What is the capital of {country}?"),
])


# 템플릿에 변수 값 주입
chat_prompt = chat_prompt_template.invoke({"country": "France"})

print(chat_prompt)

messages=[SystemMessage(content='You are a helpful assistant!', additional_kwargs={}, response_metadata={}), HumanMessage(content='What is the capital of France?', additional_kwargs={}, response_metadata={})]


In [17]:
# 생성된 메시지 확인
chat_prompt.messages

[SystemMessage(content='You are a helpful assistant!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='What is the capital of France?', additional_kwargs={}, response_metadata={})]

In [18]:
# LLM에 프롬프트 전달
llm.invoke(chat_prompt)

AIMessage(content='The capital of France is Paris.', additional_kwargs={}, response_metadata={'model': 'llama3.2:1b', 'created_at': '2025-02-02T13:25:02.510889Z', 'done': True, 'done_reason': 'stop', 'total_duration': 198474709, 'load_duration': 30474500, 'prompt_eval_count': 38, 'prompt_eval_duration': 70000000, 'eval_count': 8, 'eval_duration': 96000000, 'message': Message(role='assistant', content='', images=None, tool_calls=None)}, id='run-6a603e21-ee48-436c-bd74-dfe1239f02f6-0', usage_metadata={'input_tokens': 38, 'output_tokens': 8, 'total_tokens': 46})